<a href="https://colab.research.google.com/github/WSH032/wd-v1-4-tagger-feature-extractor-tutorials/blob/main/wd14_tf2onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装依赖

!pip install onnx tf2onnx tensorflow huggingface_hub onnxruntime -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [2]:
!pip show onnx
!pip show tf2onnx
!pip show tensorflow

Name: onnx
Version: 1.14.0
Summary: Open Neural Network Exchange
Home-page: https://github.com/onnx/onnx
Author: ONNX
Author-email: onnx-technical-discuss@lists.lfaidata.foundation
License: Apache License v2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, protobuf, typing-extensions
Required-by: tf2onnx
Name: tf2onnx
Version: 1.14.0
Summary: Tensorflow to ONNX converter
Home-page: https://github.com/onnx/tensorflow-onnx
Author: ONNX
Author-email: onnx-technical-discuss@lists.lfaidata.foundation
License: Apache License v2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: flatbuffers, numpy, onnx, requests, six
Required-by: 
Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-

In [3]:
# 下载模型

import os

from huggingface_hub import hf_hub_download


# wd-v1-4-swinv2-tagger-v2 / wd-v1-4-vit-tagger / wd-v1-4-vit-tagger-v2/ wd-v1-4-convnext-tagger / wd-v1-4-convnext-tagger-v2 / wd-v1-4-convnextv2-tagger-v2 / SmilingWolf/wd-v1-4-moat-tagger-v2
DEFAULT_WD14_TAGGER_REPO = "SmilingWolf/wd-v1-4-moat-tagger-v2"
FILES = ["keras_metadata.pb", "saved_model.pb", "selected_tags.csv"]
SUB_DIR = "variables"
SUB_DIR_FILES = ["variables.data-00000-of-00001", "variables.index"]
CSV_FILE = FILES[-1]

download_dir = "wd14_tagger_model" # 模型保存目录

def download_keras_model(repo_id: str, download_dir: str) -> None:
    print(f"downloading wd14 tagger model from hf_hub. id: {repo_id}")
    for file in FILES:
        hf_hub_download(repo_id, file, force_download=True, local_dir=download_dir)
    for file in SUB_DIR_FILES:
        hf_hub_download(
            repo_id,
            file,
            subfolder=SUB_DIR,
            force_download=True,
            local_dir=download_dir,
        )

download_keras_model(DEFAULT_WD14_TAGGER_REPO, download_dir)


downloading wd14 tagger model from hf_hub. id: SmilingWolf/wd-v1-4-moat-tagger-v2


In [4]:
# 转换模型

import time

from tensorflow.keras.models import load_model, Model
import tensorflow as tf
import tf2onnx


opset = 17  # opset运算符版本
MULTI_OUTPUT_NUMBER = 4  # 调整后输出层的数量，从最外层开始向内
input_layer_name = "input_1" # 原模型输入层名字，这里显式的指明出来

model_type = DEFAULT_WD14_TAGGER_REPO.split("/")[1]
div_str = "_"  # 分割符
ONNX_NAME = f"{model_type}{div_str}opset{opset}.onnx"  # 转换输出的onnx模型的名字


def change_keras_to_onnx(
    input_keras_model_dir,
    output_onnx_model_path: str,
    multi_output_number: int,
) -> None:

    time_start = time.time()
    print("开始将kera模型转换成onnx格式，请耐心等待...")

    # 载入模型
    print("载入keras模型...")
    keras_model = load_model(input_keras_model_dir)
    print("载入keras模型完成")

    # 获取中间层
    # layer0 = keras_model.get_layer(index=-1) # 获取最后一层的对象，predictions_sigmoid 预测层
    # layer1 = keras_model.get_layer(index=-2) # 获取倒数第二层的对象
    # layer2 = keras_model.get_layer(index=-3) # 获取倒数第三层的对象，predictions_norm 层
    # layer3 = keras_model.get_layer(index=-4) # 获取倒数第四层的对象
    multi_output_list = []
    for i in range(multi_output_number):
        multi_output_list.append( keras_model.get_layer(index = -i-1) )


    # 重构为多层输出
    multi_ouput_model = Model(
        inputs=keras_model.input,
        outputs=[layer.output for layer in multi_output_list]
    )

    # 构建onnx输入层
    multi_ouput_model_input = multi_ouput_model.input  # 原版的第一个dim=None，可以无限制batch，这里就不用改了
    # name=multi_ouput_model_input.name  # 似乎不指定他也能自己推断出来
    spec = [tf.TensorSpec(
        shape = multi_ouput_model_input.shape,
        dtype = multi_ouput_model_input.dtype,
        name = input_layer_name,
        )
    ]

    # 转换为onnx模型
    print("开始转换为onnx模型...")
    tf2onnx.convert.from_keras(
        multi_ouput_model,
        input_signature=spec,  # input_signature=None # 不指定似乎他也能推断出来
        output_path=output_onnx_model_path,
        opset=opset,
    )
    print(f"转换完成，总共用时{time.time() - time_start}秒")

change_keras_to_onnx(download_dir, ONNX_NAME, 4)


开始将kera模型转换成onnx格式，请耐心等待...
载入keras模型...


载入keras模型完成
开始转换为onnx模型...
转换完成，总共用时95.08833980560303秒


In [5]:
# 检查模型完整性

import onnx

# 这一步应该无任何输出或者报错
onnx.checker.check_model(ONNX_NAME)


In [6]:
# 查看onnx模型opset版本

model = onnx.load(ONNX_NAME)
domains = model.opset_import
for dom in domains:
    print("domain: %r, version: %r" % (dom.domain, dom.version))


domain: '', version: 17
domain: 'ai.onnx.ml', version: 2


In [7]:
# 检查转换后输入输出层名字，并载入模型

import onnxruntime

ort_session = onnxruntime.InferenceSession(ONNX_NAME)
keras_model = load_model(download_dir)

inputs_name_list = [x.name for x in ort_session.get_inputs()]
outputs_name_list = [x.name for x in ort_session.get_outputs()]

print("输入", inputs_name_list)  # 输入层名字
print("输出", outputs_name_list)  # 输出层名字


输入 ['input_1']
输出 ['predictions_sigmoid', 'predictions_dense', 'predictions_norm', 'predictions_globalavgpooling']


In [8]:
# 检查转换精度损失

import numpy as np

# 生成随机矩阵
tset_np = np.random.randint(0, 226, size=(1, 448, 448, 3)).astype(np.float32)

# keras输出
keras_output = keras_model(tset_np)

# onnx输出
ort_inputs = {inputs_name_list[0]: tset_np.astype(np.float32)}
ort_output = ort_session.run(outputs_name_list, ort_inputs)

# 精度检查，认为在1e-3误差可以接受（实际上误差可达1e-6）
# 这一步应该无任何输出、报错
np.testing.assert_allclose(ort_output[0], keras_output, atol=1e-3, rtol=0)


In [ ]:
# 上传模型

repo_id = "your_repo_id"
token = "your_token"

from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj=ONNX_NAME,
    path_in_repo=ONNX_NAME,
    repo_id=repo_id,
    token=token,
)